In [46]:
from gurobipy import *
import random as rd
import json

In [47]:
# Opening JSON file
f = open('/Users/cha/Desktop/Code/optimization-project/data/small.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
data.keys()
f.close()

In [48]:
data['staff']

[{'name': 'Olivia', 'qualifications': ['A', 'B', 'C'], 'vacations': []},
 {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]},
 {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}]

In [49]:
data['jobs']

[{'name': 'Job1',
  'gain': 20,
  'due_date': 3,
  'daily_penalty': 3,
  'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}},
 {'name': 'Job2',
  'gain': 15,
  'due_date': 3,
  'daily_penalty': 3,
  'working_days_per_qualification': {'A': 1, 'B': 2}},
 {'name': 'Job3',
  'gain': 15,
  'due_date': 4,
  'daily_penalty': 3,
  'working_days_per_qualification': {'A': 1, 'C': 2}},
 {'name': 'Job4',
  'gain': 20,
  'due_date': 3,
  'daily_penalty': 3,
  'working_days_per_qualification': {'B': 2, 'C': 1}},
 {'name': 'Job5',
  'gain': 10,
  'due_date': 5,
  'daily_penalty': 3,
  'working_days_per_qualification': {'C': 2}}]

- Xi,j,k,t ∈ {0, 1} vaut 1 si la personne i r´ealise une qualification q pour le projet j pendant la
journ´ee t, 0 sinon, pour i ∈ S, j ∈ J , k ∈ Q, t ∈ H.
- Yj ∈ {0, 1} vaut 1 si le projet j est réalisé totalement, 0 sinon, j ∈ J
- Lj nombre de jours de retard pour le projet j ∈ J
- Ej date de fin de réalisation du projet j ∈ J

In [50]:
# Instanciation du modèle
m = Model("Simple PL modelling")

# Création de variables
with open('/Users/cha/Desktop/Code/optimization-project/data/small.json') as f:
    data = json.load(f)

S = data['staff']
J = data['jobs']
Q = data['qualifications']
H = data['horizon']

X = m.addMVar(shape=(len(S), len(J), len(Q), H), vtype=GRB.BINARY)
Y = m.addMVar(shape=len(J), vtype=GRB.BINARY)
L = m.addMVar(shape=len(J))
E = m.addMVar(shape=len(J))
# maj du modèle
m.update()

In [52]:
def c1(x, i, t):
    res = 0
    for j, k in zip(range(len(J)), range(len(Q))):
        res += x[i, j, k, t]
    return res

def c4(x, j, k):
    res = 0
    for i, t in zip(range(len(S)), range(H)):
        res += x[i, j, k, t]
    return res

def obj(Y, L):
    res = 0
    for j in range(len(J)):
        job = J[j]
        gj = job["gain"]
        cj = job["daily_penalty"]
        res += (Y[j] * gj - L[j] * cj)
    return res


# C1
for i, t in zip(range(len(S)), range(H)):
    m.addConstr(c1(X, i, t) <= 1)

# C2
for i in range(len(S)):
    person = S[i]
    for t in person['vacations']:
        m.addConstr(c1(X, i, t) == 0)

# C3
for i in range(len(S)):
    person = S[i]
    for j in range(len(J)):
        job = J[j]
        for k in range(len(Q)):
            if k not in person['qualifications'] and k not in list(job['working_days_per_qualification'].keys()):
                for t in range(H):
                    m.addConstr(X[i, j, k, t] == 0)

# C4
for j in range(len(J)):
    job = J[j]
    qualifications = list(job['working_days_per_qualification'].keys())
    for k in range(len(qualifications)):
        njk = job['working_days_per_qualification'][qualifications[k]]
        m.addConstr(Y[j]*njk <= c4(X, j, k))

# C5
for j in range(len(J)):
    job = J[j]
    qualifications = list(job['working_days_per_qualification'].keys())
    for k in range(len(qualifications)):
        njk = job['working_days_per_qualification'][qualifications[k]]
        m.addConstr(c4(X, j, k) <= njk)

# C6
for i in range(len(S)):
    for j in range(len(J)):
        for k in range(len(Q)):
            for t in range(H):
                m.addConstr(X[i, j, k, t]*t <= E[j])

# C7
for j in range(len(J)):
    job = J[j]
    dj = job["due_date"]
    m.addConstr(E[j] - dj <= L[j])

# Ej <= H
for j in range(len(J)):
    m.addConstr(E[j] <= H)

# Fonction Objectif
m.setObjective(
    obj(Y, L),
    GRB.MAXIMIZE
)   
# Paramétrage (mode mute)
m.params.outputflag = 0

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [54]:
# Résolution du PL
m.optimize()
print("La solution optimale est (X, Y, L, E) = {} avec pour objectif z = {}".format((X.X, Y.X, L.X, E.X), m.objVal))

La solution optimale est (X, Y, L, E) = (array([[[[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]]],


       [[[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0., -0., -0., -0.],
     

## Tuto internet

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

In [ ]:
# Create a new model
m = gp.Model("matrix1")

# Create variables
x = m.addMVar(shape=3, vtype=GRB.BINARY, name="x")

# Set objective
obj = np.array([1.0, 1.0, 2.0])
m.setObjective(obj @ x, GRB.MAXIMIZE)

# Build (sparse) constraint matrix
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

# Build rhs vector
rhs = np.array([4.0, -1.0])

# Add constraints
m.addConstr(A @ x <= rhs, name="c")

# Optimize model
m.optimize()

print("here", x.X)
print('Obj: %g' % m.ObjVal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x8d4960d3
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
here [1. 0. 1.]
Obj: 3
